In [ ]:
import io
import zipfile

import pandas as pd
import requests

from utils import settings

In [ ]:
# This will need to be updated once Clarity starts posting results for the race...
# Typically this is right after the polls close in Georgia.
SUMMARY_CSV_URL = 'https://results.enr.clarityelections.com//GA//107395/272627/reports/summary.zip'

In [ ]:
r = requests.get(SUMMARY_CSV_URL)

In [ ]:
r.status_code

In [ ]:
content = zipfile.ZipFile(io.BytesIO(r.content))

In [ ]:
m = content.infolist()[0].date_time

In [ ]:
last_updated = f'{m[0]}-{m[1]}-{m[2]} {m[3]}:{m[4]}:{m[5]}.000000'

In [ ]:
summary_results = pd.read_csv(io.BytesIO(content.read('summary.csv')), encoding='latin1')

In [ ]:
column_rename = {
    "contest name": "clarity_office",
    "choice name": "clarity_candidate",
    "total votes": "votes",
    "percent of votes": "percent_votes",
}
summary_results.rename(columns=column_rename, inplace=True)

In [ ]:
summary_results.drop(
    columns=['line number', 'party name', 'num Precinct total', 'num Precinct rptg', 'over votes', 'under votes'], 
    inplace=True
)

In [ ]:
summary_results.head()

In [ ]:
summary_results.info()

### Use this next cell to lookup Clarity information...

In [ ]:
pd.set_option('max_colwidth', None)
summary_results[summary_results['clarity_candidate'].str.contains("Pritchett")].T

In [ ]:
branch_candidate_office_20201201 = [
    {
        'branch_id': 'linda-pritchett',
        'branch_candidate': 'Linda Pritchett',
        'branch_race_id': 'ga-general-runoff-2020-ga-senate-39',
        'clarity_candidate': 'Linda Pritchett (DEM)',
        'clarity_office': 'State Senate District 39 - Special Democratic Primary'
    },
    {
        'branch_id': 'sonya-halpern',
        'branch_candidate': 'Sonya Halpern',
        'branch_race_id': 'ga-general-runoff-2020-ga-senate-39',
        'clarity_candidate': 'Sonya Halpern (DEM)',
        'clarity_office': 'State Senate District 39 - Special Democratic Primary'
    },
    {
        
        'branch_id': 'kwanza-hall',
        'branch_candidate': 'Kwanza Hall',
        'branch_race_id': 'ga-special-2020-us-house-5',
        'clarity_candidate': 'Kwanza Hall',
        'clarity_office': 'US House District 5'
    },
    {
        'branch_id': 'robert-franklin',
        'branch_candidate': 'Robert Franklin',
        'branch_race_id': 'ga-special-2020-us-house-5',
        'clarity_candidate': 'Robert Franklin',
        'clarity_office': 'US House District 5'
    },
]

In [ ]:
branch_info = pd.DataFrame(branch_candidate_office_20201201)

In [ ]:
branch_info.info()

In [ ]:
pd.set_option('max_colwidth', None)
branch_info.head()

In [ ]:
type(branch_info)

In [ ]:
pd.set_option('max_colwidth', None)
branch_info.head().T

In [ ]:
merged_results = pd.merge(branch_info, summary_results, on=['clarity_candidate', 'clarity_office'], how='inner')

In [ ]:
merged_results.head()

In [ ]:
type(merged_results)

In [ ]:
merged_results.info()

In [ ]:
type(merged_results)

In [ ]:
merged_results.insert(0, 'last_updated', last_updated)

In [ ]:
merged_results.head()

In [ ]:
election_results_csv = io.StringIO()
columns = [
    'last_updated',
    'branch_id',
    'branch_race_id',
    'clarity_candidate',
    'clarity_office',
    'votes',
    'percent_votes'
]
merged_results.to_csv(election_results_csv, columns=columns, header=True, index=False)

In [ ]:
election_results_csv.getvalue()

In [ ]:
url = settings.BRANCH_URL_AUTH

headers = {
    "Content-Type": "application/json",
}

data = {
    "email": settings.BRANCH_EMAIL,
    "password": settings.BRANCH_PASSWORD,
    "strategy": settings.BRANCH_STRATEGY,
}

In [ ]:
results = requests.post(url, json=data, headers=headers)
results.status_code

In [ ]:
branch_token = results.json().get('accessToken')

In [ ]:
url = settings.BRANCH_URL

headers = {
    'Authorization': branch_token
}

files = {'file': election_results_csv.getvalue()}

In [ ]:
r = requests.post(url, files=files, headers=headers)
r.status_code

In [ ]:
r.json().get('location')